In [107]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
import numpy as np
import matplotlib.pyplot as plt
_include_('curriculum_vqa')
from cvqa import teacher


In [124]:
from cvqa.curriculum import plotter, VQAInstanceDistribution

vqa_dist = VQAInstanceDistribution()
vizrep = vqa_dist.sample_viz_rep()
s = vqa_dist.sample_prompt(vizrep)

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(vizrep)
pp.pprint(s)

{'color': 'blue', 'location': [0.35, 0.73], 'shape': 'circle', 'size': 0.22}
[   {   'concept': 'color',
        'prompt': 'The color of the object in the picture is...?',
        'target': 'blue'}]


In [140]:
vizreps = []
for i in range(7):
    vizreps.append(vqa_dist.sample_viz_rep())

In [141]:
from cvqa import models

m = models.StructuredImageModel(vizreps, 5, output_channels=6)
m.features

{'color': {'blue': 0, 'grey': 1, 'red': 2},
 'shape': {'circle': 0, 'triangle': 1}}

In [142]:
m.feat_tensors

{'color': {'blue': tensor([[1, 0, 0, 0, 0]]),
  'grey': tensor([[0, 1, 0, 0, 0]]),
  'red': tensor([[0, 0, 1, 0, 0]])},
 'shape': {'circle': tensor([[1, 0, 0, 0, 0]]),
  'triangle': tensor([[0, 1, 0, 0, 0]])}}

In [143]:
import torch
import torch.nn.functional as F

oh = F.one_hot(torch.arange(5))
oh

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])

In [144]:
for f in m.features:
    vals = m.features[f]
    for v in vals:
        print(oh[vals[v]])

tensor([1, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 0])
tensor([0, 0, 1, 0, 0])
tensor([1, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 0])


In [145]:
m(None, structured_reps=vizreps).shape

torch.Size([7, 6, 5])